In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 125

import seaborn as sns

import pandas as pd
import numpy as np
np.random.seed(0) # Giúp chạy các hàm của scikit-learn giống nhau mỗi lần chạy

import regex as re
import time # Dùng để sleep chương trình
from tqdm.notebook import tqdm # Hiện thanh progress cho đẹp :D
tqdm.pandas()

# Thư viện để request và parse HTML
import requests
from bs4 import BeautifulSoup

# Các thư viện liên quan tới ngôn ngữ và NLP
from pyvi import ViTokenizer # Thư viện NLP tiếng Việt
import gensim
import unicodedata # Thư viện unicode

# Trực quan hóa mô hình dự đoán văn bản
from lime import lime_text

# Dùng để lưu lại model
import pickle

# Thư viện liên quan của Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import feature_selection

# Tạo pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer, make_column_selector

# Các mô hình học
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier # Phương pháp bagging

In [1]:
# Thiết lập đường dẫn cho phần 1
dir_1 = "C:/Users/davin/Desktop/KHDL/"
# Thiết lập lại đường dẫn phần 1
dir_1_new = "C:/Users/davin/Desktop/KHDL/raw_data/"

In [8]:
# HÀM REQUEST CONTENT
def single_request_scraping(index = 1, limit_retry = 100, sleep_time = 0.05):

    #THIẾT LẬP YÊU CẦU HTTP
    allow_status = [200, 301]
    # Tạo một phiên làm việc bằng requests.Session()
    s = requests.Session()
    #cấu hình User-Agent giả lập trình duyệt, giảm nguy cơ bị chặn từ phía trang web
    s.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    url = f"https://tuoitre.vn/timeline/0/trang-{index}.htm"
    
    
    # LẤY DANH SÁCH NEWS_ITEMS
    try:
    #Gửi yêu cầu GET đến url
        response = s.get(url)
        try_left = limit_retry
        while (response.status_code not in allow_status and try_left > 0):
            print(f"Loi {response.status_code} tai trang {index}")
            response = s.get(url)
            try_left -= 1
        html_text = response.text
    except:
        print(f"Loi request tai trang {index}")
        return None
        
    
    # PHÂN TÍCH HTML VÀ CHUYỂN ĐỔI HTML_TEXT THÀNH ĐỐI TƯỢNG TREE HỖ TRỢ TRUY SUẤT CÁC PHẦN TỬ HTML
    html_tree = BeautifulSoup(html_text, 'html.parser')
    news_items = html_tree.find_all('div', class_='box-category-item')
    
    #Kiểm tra lỗi và số item
    if (len(news_items) == 0):
        print(f'Trang {index} khong lay duoc item.')
        print(html_text)
        return None
    elif (len(news_items) != 20):
        print(f"Warning: Trang {index} chi lay duoc {len(news_items)} items.")
    
    
    # LẤY CÁC DỮ LIỆU LINK, TITLE, DESCRIPTION, CLASS
    # Tạo danh sách để lưu dữ liệu
    single_data = []
    # Lấy ra link, title, description và category từ mỗi bài báo
    for item in news_items:
            
        # Lấy title và link
        link_tag = item.find('a', class_='box-category-link-title')
        if link_tag:
            title = link_tag.get('title', '').strip()
            link = "https://tuoitre.vn" + link_tag.get('href', '')
        
        # Lấy category
        category_tag = item.find('a', class_='box-category-category')
        category = category_tag.text.strip() if category_tag else ""
        
        # Lấy description
        description_tag = item.find('p', class_='box-category-sapo')
        description = description_tag.text.strip() if description_tag else ""
        
        # Thêm dữ liệu vào danh sách
        single_data.append({"links": link, "title": title, "description": description, "class": category})
    
    raw_data = pd.DataFrame(single_data)
    
    # LẤY DỮ LIỆU CONTENT
    raw_data["content"] = ""
    
    #phân tích HTML
    for tmp_index, row in raw_data.iterrows():
        try:
            response = s.get(row["links"])
            try_left = limit_retry
            while (response.status_code not in allow_status and try_left > 0):
                print(f"Loi {response.status_code} tai link {row['links']}")
                response = s.get(row["links"])
                try_left -= 1
            news_page = response.content
        except:
            print(f"Loi request tai link {row['links']}")
            
    #chuyển đổi html_text thành đối tree giúp truy xuất các phần tử trong HTML
        tmp_tree = BeautifulSoup(news_page, "html.parser")
    
    # Lấy nội dung
        try:
            body = tmp_tree.find("div", class_="detail-content afcbc-body")
            content = body.findChildren("p", recursive=False)
            tmp_string = ""
            for x in content:
                tmp_string += x.text
                raw_data.at[tmp_index, "content"] = tmp_string
        except:
            raw_data.at[tmp_index, "content"] = ""
    # Thời gian nghỉ tránh bị khoá ip
        time.sleep(sleep_time)
    return raw_data

In [ ]:
#Batch scraping
def batch_scraping(num = 200, output_dir = ""):
    '''
        num: số lượng trang request cho 1 batch (1 trang = 20 bài báo). Mặc định 4000 bài báo.
    '''
    iter_num = 1  # Số batch bắt đầu
    continue_flag = True # Cờ hiệu kết thúc vòng lặp khi xảy ra lỗi

    while (continue_flag):
        '''
        Khởi tạo dataframe rỗng. 
        Sau đó lấy đủ 1 số trang cho 1 batch.
        Rồi export file csv.
        '''
        
        batch_df = pd.DataFrame(columns=["links", "title", "description", "class", "content"])
        
        for index in range(iter_num * num + 1, (iter_num + 1) * num + 1):
            data = single_request_scraping(index)
            if (data is None):
                continue_flag = False
                break
            print(f"Page {index} complete!")
            batch_df = pd.concat([batch_df, data], ignore_index=True)
        batch_df.to_csv(output_dir + f'crawling_{iter_num}.csv',index=False,encoding="utf-8")
        iter_num+=1
        
batch_scraping(output_dir = dir_1)